## Video Load

In [11]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from util import *

In [12]:
# 영상 FPS, 재생 속도, 프레임 스킵 수 설정
fps, speed, skip = 30, 1, 0

def getFrameDuration():
    return 1 / fps * (skip + 1)

# 영상 사이즈 기준 1920 x 1080
# 영상 크롭 및 사이즈 조정 (x1/4, x1/16)
class Frame:
    def __init__(self, frame, crop=64):
        self.x04 = cv2.resize(frame[crop:-crop], (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        self.x16 = cv2.resize(self.x04,          (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

        x04_hsv = cv2.split(cv2.cvtColor(self.x04, cv2.COLOR_BGR2HSV))
        x16_hsv = cv2.split(cv2.cvtColor(self.x16, cv2.COLOR_BGR2HSV))
        
        self.x04_H, self.x04_S, self.x04_V = x04_hsv
        self.x16_H, self.x16_S, self.x16_V = x16_hsv

        self.x04_field_mask, self.x04_player_mask = None, None

In [13]:
class HistogramAnalysis:
    def __init__(self, frame):
        self.frame = frame

        # 히스토그램 분석
        self.hist = cv2.calcHist([frame], [0], None, [256], [0, 256])
        
        # 히스토그램 확률 밀도 함수
        self.hist = self.hist.squeeze() / self.hist.sum()

        # 히스토그램 평균
        self.mean = np.mean(self.hist * np.arange(256)) * 255

        # 히스토그램 분산 및 표준 편차
        var = self.hist * ((np.arange(256) - self.mean) ** 2)

        self.var = np.sum(var)

        self.std = ((math.sqrt(np.sum(var[:int(self.mean)]) * 2), math.sqrt(np.sum(var[int(self.mean):]) * 2)))

In [14]:
diff_graph, changed_graph, OOField_graph, scene_color_graph = None, None, None, None

gap, scene_index, scene_color = 0, 0, [255, 255, 255]

def init_scene_detection():
    global diff_graph, changed_graph, OOField_graph, scene_color_graph, gap, scene_index, scene_color 
    
    diff_graph, changed_graph, OOField_graph, scene_color_graph = [0] * 256, [False] * 256, [True] * 256, [[255, 255, 255]] * 256

    gap, scene_index, scene_color = 0, -1, [255, 255, 255]
    
def process_scene_change(prev : Frame, frame : Frame, hist_analyzed : HistogramAnalysis, debug=False):
    global scene_index, scene_color, gap

    # 이전 프레임과의 변화량 계산
    diff = int(np.sum(cv2.absdiff(prev.x16, frame.x16))) / frame.x16.size

    changed = abs(diff - diff_graph[-1]) > 10 * (1 + math.log(skip + 1) * 0.6) and gap >= 0.5

    sample_count = 10

    l, r = hist_analyzed.std

    # 샘플된 프레임 변화량 평균이 일정 값 이상인지 체크한다
    isDiffover = (diff + np.sum(diff_graph[-sample_count:])) / sample_count > 3
    
    # 히스토그램의 분포를 이용해서 필드인지 아닌지를 분류한다.
    isFocusing = hist_analyzed.mean + r > 140
    
    # 몇 프레임 이전에 화면 전환이 이루어졌거나 아니면 이미 필드 밖이라면 True가 된다.
    hasChanged = (np.array(changed_graph[-sample_count:]).any() or np.array(OOField_graph[-sample_count:]).any())

    # 최종적으로 화면이 필드가 아닌지를 체크하는 플래그를 계산한다.
    OOField = (isDiffover or isFocusing) and hasChanged
    
    # 장면 전환이 이루어졌을때 히스토그램 인덱스 및 색상 처리
    if OOField and not OOField_graph[-1]:
        scene_color = [0, 0, 255]

    if not OOField and OOField_graph[-1]:
        scene_index += 1

        if debug: print("Scene #{0:05d}".format(scene_index))

        scene_color = np.random.randint([128, 128, 128], 255, size=3)

    # 값을 저장해서 다음 프레임 계산에 사용한다.
    diff_graph.append(diff) 
    diff_graph.pop(0)

    gap += getFrameDuration()
    
    if(changed): gap = 0
    
    changed_graph.pop(0) 
    changed_graph.append(changed)
    
    OOField_graph.pop(0) 
    OOField_graph.append(OOField)

    scene_color_graph.pop(0) 
    scene_color_graph.append(scene_color)
    
    # ===== 이하 디버깅을 위한 히스토그램 표시 =====
    if debug:        
        hist_img = get_histogram_image(np.array(diff_graph))

        for i in range(256):
            hist_img[:, i, :] = np.bitwise_and(hist_img[:, i, :], scene_color_graph[i])
        
        cv2.imshow("Scene Diffs", hist_img)
    # ===========================================

def is_frame_on_field():
    return not OOField_graph[-1]

In [15]:
def analyze_field_feature(hist_analyzed : HistogramAnalysis, debug=False): 
    # 분석한 히스토그램을 바탕으로 필드 영역 추출
    l_weight, r_weight = 1.35, 0.5

    min_range, max_range = 32, 72

    std_l, std_r = hist_analyzed.std

    l, r = max(int(hist_analyzed.mean - std_l * l_weight), min_range), min(int(hist_analyzed.mean + std_r * r_weight), max_range)

    # ===== 이하 디버깅을 위한 히스토그램 표시 =====
    if debug:
        hist_img = get_histogram_image(hist_analyzed.hist)

        line_histogram_image(hist_img, l, (0, 0, 255))
        line_histogram_image(hist_img, r, (0, 0, 255))
        line_histogram_image(hist_img, int(hist_analyzed.mean), (255, 0, 255))

        cv2.imshow("Histogram Field", hist_img) 
    # ===========================================

    return (l, r)

In [16]:
def extract_field_mask(hist_analyzed : HistogramAnalysis):
    field_feature_analyzed = analyze_field_feature(hist_analyzed, debug=True)

    field_feature = cv2.inRange(hist_analyzed.frame, *field_feature_analyzed)
    
    kernel = np.ones((13, 13))
    field_feature = cv2.erode(field_feature, kernel)

    kernel = np.ones((9, 9))
    field_feature = cv2.dilate(field_feature, kernel)  

    kernel = np.ones((3, 3))
    field_feature = cv2.erode(field_feature, kernel)  

    # 필드 영역 마스크 생성
    field_mask = np.zeros(field_feature.shape, dtype=np.uint8)
    
    # 외곽선을 찾아서 필드 영역을 단순화
    contours, hierarchy = cv2.findContours(field_feature, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour_length = []

    for contour in contours:
        contour_length.append(cv2.arcLength(contour, True))

    if len(contour_length) > 0:
        contour = contours[int(np.argmax(contour_length))]

        cv2.drawContours(field_mask, [cv2.convexHull(contour)], -1, 255, -1) 

    return field_mask

In [17]:
def extract_player_mask(frame: Frame, hist_analyzed : HistogramAnalysis):
    field_feature_analyzed = analyze_field_feature(hist_analyzed, debug=False)

    field_feature = cv2.inRange(hist_analyzed.frame, *field_feature_analyzed)

    kernel = np.ones((5, 5))
    field_mask = cv2.erode(frame.x04_field_mask, kernel)  

    w_or_b_s = cv2.inRange(frame.x04_S, 0, 60)
    w_or_b_v = cv2.inRange(frame.x04_V, 0, 45)

    w_or_b = cv2.bitwise_or(w_or_b_s, w_or_b_v)

    player_mask = cv2.bitwise_not(field_feature)
    
    player_mask = cv2.bitwise_or(w_or_b, player_mask)

    player_mask = cv2.copyTo(player_mask, field_mask)

    kernel = np.ones((3, 3))
    player_mask = cv2.dilate(player_mask, kernel)  

    return player_mask

In [18]:
def analyze(prev : Frame, frame : Frame):
    hist_analyzed = HistogramAnalysis(frame.x04_H)

    process_scene_change(prev, frame, hist_analyzed, debug=True)
    
    if is_frame_on_field():
        frame.x04_field_mask    = extract_field_mask(hist_analyzed)
        frame.x04_player_mask   = extract_player_mask(frame, hist_analyzed)

        cv2.imshow("Frame",  frame.x04)
        cv2.imshow("Mask", cv2.hconcat((frame.x04_field_mask, frame.x04_player_mask)))

In [19]:
def start_analyzer(path):
    init_scene_detection()
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)

    if capture.isOpened():
        prev, (run, frame) = None, capture.read()  # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            prev = frame = Frame(frame)

    current_skip = 0

    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우  
            if current_skip == 0:
                frame = Frame(frame)
                
                situation = analyze(prev, frame)

                cv2.imshow("Frame", frame.x04) 
                cv2.waitKey(max(int(1000 / (fps * speed)), 1)) # Millisecond 단위로 대기
            
                prev = frame
            
            current_skip = (current_skip + 1) % (skip + 1)
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break

    capture.release() # Capture 자원 반납 
    cv2.destroyAllWindows() # 창 제거

In [20]:
# Video가 저장된 경로 입력
PATH = r"video/soccer2.mp4"

# Video 재생 및 반환 (Numpy Array)
fps, speed, skip = 30, 4, 0
start_analyzer(PATH)

Scene #00000
Scene #00001
Scene #00002
Scene #00003
Scene #00004
Scene #00005
Scene #00006
Scene #00007


In [21]:
cv2.destroyAllWindows() # 창 제거

            #height, width, channel = frame.shape
            #cos = -int(math.cos(math.radians(angle)) * 1000)
            #sin = int(math.sin(math.radians(angle)) * 1000)
            
            #cv2.line(frame, (-cos+width//2, -sin+height//2), (cos+width//2, sin+height//2), (0, 0, 255), 1)
            
            #cv2.line(frame, (width // 2, height // 2), (width // 2, height // 2), (0, 0, 255), 5)
            #cv2.rectangle(frame, (width // 2 - height // 2, 0), (width // 2 + height // 2, height), (0, 255, 255), 10)
            #frame = cv2.copyTo(frame, field_mask)